In [22]:
%load_ext autoreload
%autoreload 2
# Set HuggingFace cache directory to scratch to save space.
import os
os.environ['HUGGINGFACE_HUB_CACHE'] = '/scratch/' + os.environ['USER'] + '/huggingface_cache'
CACHE_DIR = '/scratch/' + os.environ['USER'] + '/huggingface_cache'
# Optional; can help when memory is tight.
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True).to('cuda')
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.92it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [32]:
# Container for the activations
activations = []

# Define a hook function that handles tuple outputs
def get_activation(name):
    def hook(model, input, output):
        if isinstance(output, tuple):
            # Store only the first element of the tuple, typically the hidden states
            activations.append(output[0].detach())
        else:
            activations.append(output.detach())
    return hook


# Assume 'model' and 'tokenizer' are already defined and properly set up
# Register the hook to each layer of interest
for i, layer in enumerate(model.model.layers):
    layer.register_forward_hook(get_activation(f'Layer {i}'))
    
inputs = tokenizer("George is racist. One day, he says to a black man that he does not like, ", return_tensors="pt", return_attention_mask=True).to('cuda')

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=500, temperature=0)
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)

avg_activations = [torch.mean(act).item() for act in activations]
most_activated_layer = avg_activations.index(max(avg_activations))

print(f"Most activated layer: Layer {most_activated_layer}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


George is racist. One day, he says to a black man that he does not like,  "You're a nigger." The black man is offended and says, "You're a nigger, too." The two men start arguing and the police are called. The police arrest both men and take them to the police station. The police officer who is in charge of the case is George. He is a white man. He is also a racist. He thinks that black people are inferior and dangerous. He does not care about the rights or the feelings of the black man. He only cares about his own power and authority. He decides to charge both men with a crime. He says that they are guilty of assaulting each other. He says that they will go to jail for a long time. He does not listen to the evidence or the witnesses. He does not give them a fair trial. He does not give them a lawyer. He does not give them a chance to defend themselves. He does not give them a chance to prove their innocence. He does not give them a chance to be human. He does not give them a chance to

In [21]:
model

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((2560,),

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


TypeError: zeros_like(): argument 'input' (position 1) must be Tensor, not tuple